Inspired by https://arxiv.org/pdf/1609.02907.pdf

Install everything in the Readme

In [1]:
import torch
import numpy as np
import scipy.sparse as sp

GCN spec:

Needs: <br>
-adjacency list of size nxn <br>
-output of size nx1 <br>
-classification or regression

Can also provide: <br>
-initial features of size nxf <br>
-hidden layer size (hl) <br>
-train/test split on nodes


TODO: <br>
-sticking it on a GPU, which really just means adding .cuda() and a function parameter <br>
-parameterize `predict` function, so you can predict all nodes or just a subset (train/test); easy to do

# Hand-written GCN class

In [2]:
class GCN:
    def __init__(self, adj_list, out_labels, method, nl=3, init_feats=None, train_test=0.5):
        """
        Creates important model variables and weights
        method: 'cat' or 'cont'
        """
        self.adj_list = torch.from_numpy(adj_list).double()
        sqrt = True # turn to False to use 'simple' adj_list normalization
        D = GCN.create_D(self.adj_list, sqrt=sqrt)
        if sqrt:
            self.norm_adj_list = D @ self.adj_list @ D # https://tkipf.github.io/graph-convolutional-networks/#fn3
        else:
            self.norm_adj_list = D @ self.adj_list
        self.out_labels = torch.from_numpy(out_labels).view(1, -1)
        self.method = method
        if self.method == 'cat':
            self.nc = len(torch.unique(self.out_labels))
        # if no features given, use identity matrix
        if init_feats is None:
            init_feats = np.eye(len(adj_list))
        self.init_feats = torch.from_numpy(init_feats).double()
        self.nl = nl
        # can give train indices directly or give a train/test split
        if type(train_test) == np.ndarray:
            self.train_indices = train_test
        else:
            self.train_indices = np.random.choice(np.arange(len(adj_list)), int(train_test*len(adj_list)), replace=False)
        
        self.weight_list = []
        self.bias_list = []
        if self.method == 'cat':
            sizes = [max(40 - 5*i, 2*self.nc) for i in range(self.nl)]
        else:
            sizes = [max(40 - 5*i, 20) for i in range(self.nl)]
        sizes[-1] = self.nc if self.method == 'cat' else 1 # 1 for continuous output
        for i in range(self.nl):
            if i == 0:
                w = torch.randn(init_feats.shape[1], sizes[0], dtype=torch.double, requires_grad=True)
            else:
                w = torch.randn(sizes[i-1], sizes[i], dtype=torch.double, requires_grad=True)
            b = torch.randn(sizes[i], dtype=torch.double, requires_grad=True)
            
            self.weight_list.append(w)
            self.bias_list.append(b)
            
        self.lr = 1e-2
        self.epoch_stats = []
        
        
    def train(self, epochs=100):
        for i in range(epochs):
            self.train_epoch()
            
            
    def train_epoch(self):
        """
        Propogates the model through the GCN using the formula:
            L_next = A_hat @ L_cur @ w_cur
            where A_hat is the normalized adj matrix
        Then, backprops the model and updates the weights
        Stores, the loss in self.epoch_stats
        """
        l_prev = None
        for i, (w,b) in enumerate(zip(self.weight_list, self.bias_list)):
            if i == 0:
                l_prev = self.gcn_layer(self.norm_adj_list, self.init_feats, w, b, 'relu')
            elif i != len(self.weight_list)-1:
                l_prev = self.gcn_layer(self.norm_adj_list, l_prev, w, b, 'relu')
            else:
                if self.method == 'cat':
                    l_prev = self.gcn_layer(self.norm_adj_list, l_prev, w, b, 'softmax') # predictions!
                else:
                    l_prev = self.gcn_layer(self.norm_adj_list, l_prev, w, b, None) # no activation for final layer
                    
        loss = self.compute_loss(l_prev, self.out_labels)
        self.epoch_stats.append(loss)
        
        # backprop
        loss.backward() # torch is amazing.. :)
        with torch.no_grad():
            for i in range(len(self.weight_list)):
                self.weight_list[i] -= self.lr * self.weight_list[i].grad # updates weight
                self.weight_list[i].grad.zero_() # resets to 0
                
                
    def predict(self):
        if self.method == 'cat':
            return self.predict_cat()
        else:
            return self.predict_cont()
        
        
    def predict_cont(self):
        with torch.no_grad():
            l_prev = None
            for i, (w, b) in enumerate(zip(self.weight_list, self.bias_list)):
                if i == 0:
                    l_prev = self.gcn_layer(self.norm_adj_list, self.init_feats, w, b, 'relu')
                elif i != len(self.weight_list)-1:
                    l_prev = self.gcn_layer(self.norm_adj_list, l_prev, w, b, 'relu')
                else:
                    l_prev = self.gcn_layer(self.norm_adj_list, l_prev, w, b, None) # predictions!
            return l_prev
        
        
    def predict_cat(self):
        with torch.no_grad():
            l_prev = None
            for i, (w, b) in enumerate(zip(self.weight_list, self.bias_list)):
                if i == 0:
                    l_prev = self.gcn_layer(self.norm_adj_list, self.init_feats, w, b, 'relu')
                elif i != len(self.weight_list)-1:
                    l_prev = self.gcn_layer(self.norm_adj_list, l_prev, w, b, 'relu')
                else:
                    l_prev = self.gcn_layer(self.norm_adj_list, l_prev, w, b, 'softmax') # predictions!
            return torch.argmax(l_prev, dim=1)
    
    
    def compute_loss(self, preds, actual):
        if self.method == 'cat':
            return self.compute_cat_loss(preds, actual)
        else:
            return self.compute_cont_loss(preds, actual)
        
        
    def compute_cont_loss(self, preds, actual):
        """
        Uses MSE loss
        """
        masked_actual = actual.clone().detach().view(-1,1)
        masked_actual[~self.train_indices,:] = 0 # if not a train index, set to 0
        preds *= (masked_actual != 0).double()
        # by doing this, we are calculating the loss for the vertices specified as everything else is 0
        # this will also make sure the loss is only fed (via sum) from rows with train nodes
        # hence, weights are only updated with regards to this loss
        # and the GCN effectively still doesn't know about the vertices it never got the actual loss for !
        return ((preds - masked_actual)**2).sum()
        
    
    def compute_cat_loss(self, preds, actual):
        """
        Uses categorical cross-entropy loss
        """
        # this assigns each class a unique index. This is helpful becase the classes could be 2 and 8
        # for all I know. This will assign 2 to 0 and 8 to 1
        classes = torch.unique(actual).numpy()
        mapping = {}
        for i in range(len(classes)):
            mapping[classes[i]] = i
        # this makes a one-hot matrix based on the number of classes
        baseline = torch.eye(self.nc)
        # this is the final one-hot, where each row corresponds to the actual output in one-hot form
        one_hot = np.zeros(preds.shape, dtype=np.double)
        # this grabs the vertices that we can actually use to train and assigns the row in 
        # one-hot to the actual one-hot value (so if we are at vertex 2 and class=3, we turn 
        # row 2 into 0 1 0 0 ... 0)
        for i, v in enumerate(actual.numpy().reshape(-1)[self.train_indices]):
            index = self.train_indices[i]
            one_hot[index,:] = baseline[mapping[v]].numpy().tolist()
        one_hot = torch.from_numpy(one_hot)
        preds = -torch.log(preds)
        # by doing this, we are calculating the loss for the vertices specified as everything else is 0
        # this will also make sure the loss is only fed (via sum) from rows with non-zero one-hot
        # aka the vertices we set in the previous for loop
        # hence, weights are only updated with regards to this loss
        # and the GCN effectively still doesn't know about the vertices it never got the actual loss for !
        return (preds * one_hot).sum()
    
        
    def create_D(adj_list, sqrt=True):
        # creates the matrix for making A_hat
        # sqrt is optional, can simply invert the diagonal as well, but the paper recommends this approach
        D = torch.eye(len(adj_list), dtype=torch.double)
        diag_sums = adj_list.sum(dim=0)
        for i in range(len(adj_list)):
            if sqrt:
                D[i,i] = 1/torch.sqrt(diag_sums[i])
            else:
                D[i,i] = 1/diag_sums[i]
        return D
    
    
    def gcn_layer(self, ad_list, inp, weights, bias, activation):
        """
        Propogates the model through the GCN using the formula:
            L_next = A_hat @ L_cur @ w_cur + bias
            where A_hat is the normalized adj matrix
        """
        out = ad_list @ inp @ weights + bias
        if activation == 'relu':
            return torch.relu(out)
        elif activation == 'softmax':
            return torch.softmax(out, dim=0)
        elif activation is None:
            return out
        else:
            raise ValueError()

# Karate Club Example
https://en.wikipedia.org/wiki/Zachary%27s_karate_club

In [3]:
import networkx

In [4]:
G = networkx.karate_club_graph()

In [5]:
adj_dict = dict(G.adjacency())

In [6]:
adj_m = np.eye(len(adj_dict.keys()))
for k in adj_dict:
    for n in adj_dict[k]:
        adj_m[k,n] = 1

In [7]:
adj_m # this has self loops, so A is a neighbor of A; this is good

array([[1., 1., 1., ..., 1., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 1., 0.],
       ...,
       [1., 0., 0., ..., 1., 1., 1.],
       [0., 0., 1., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.]])

In [8]:
out_labels = []
for d in G.nodes:
    if G.nodes[d]['club'] == 'Mr. Hi':
        out_labels.append(0)
    else:
        out_labels.append(1)
out_labels = np.array(out_labels)

In [9]:
out_labels # 0 = Mr. Hi, 1 = the other guy

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [10]:
# known = [3, 5, 24, 28]
# known = np.array(known) # can also pass in this into "train_test" to show which nodes are known

In [11]:
# giving the model about 0.3 of the data makes it predict nearly every node correctly
gcn = GCN(adj_m, out_labels, method='cat', nl=3, init_feats=None, train_test=0.2)

In [12]:
preds_init = gcn.predict(); preds_init # nonsense initial predictions

tensor([1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 1, 1])

In [13]:
out_labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [14]:
(preds_init.numpy() == out_labels).mean() # very bad clearly

0.5

In [15]:
gcn.train(epochs=40)

In [16]:
preds_init = gcn.predict()
(preds_init.numpy() == out_labels).mean() # does this well with just 0.2 of the nodes

0.7058823529411765

In [17]:
gcn.epoch_stats

[tensor(38.4060, dtype=torch.float64, grad_fn=<SumBackward0>),
 tensor(37.2317, dtype=torch.float64, grad_fn=<SumBackward0>),
 tensor(36.0834, dtype=torch.float64, grad_fn=<SumBackward0>),
 tensor(34.9536, dtype=torch.float64, grad_fn=<SumBackward0>),
 tensor(33.8325, dtype=torch.float64, grad_fn=<SumBackward0>),
 tensor(32.7415, dtype=torch.float64, grad_fn=<SumBackward0>),
 tensor(31.6803, dtype=torch.float64, grad_fn=<SumBackward0>),
 tensor(30.6531, dtype=torch.float64, grad_fn=<SumBackward0>),
 tensor(29.6572, dtype=torch.float64, grad_fn=<SumBackward0>),
 tensor(28.6935, dtype=torch.float64, grad_fn=<SumBackward0>),
 tensor(27.7612, dtype=torch.float64, grad_fn=<SumBackward0>),
 tensor(26.8718, dtype=torch.float64, grad_fn=<SumBackward0>),
 tensor(26.0283, dtype=torch.float64, grad_fn=<SumBackward0>),
 tensor(25.2252, dtype=torch.float64, grad_fn=<SumBackward0>),
 tensor(24.4649, dtype=torch.float64, grad_fn=<SumBackward0>),
 tensor(23.7506, dtype=torch.float64, grad_fn=<SumBackw

# Cora

In [18]:
def encode_onehot(labels):
    classes = set(labels)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in
                    enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)),
                             dtype=np.int32)
    return labels_onehot

def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx

In [19]:
# from https://github.com/tkipf/gcn
def load_data(path="data/cora/", dataset="cora"):
    """Load citation network dataset (cora only for now)"""
    print('Loading {} dataset...'.format(dataset))

    idx_features_labels = np.genfromtxt("{}{}.content".format(path, dataset),
                                        dtype=np.dtype(str))
    features = sp.csr_matrix(idx_features_labels[:, 1:-1], dtype=np.float32)
    labels = encode_onehot(idx_features_labels[:, -1])

    # build graph
    idx = np.array(idx_features_labels[:, 0], dtype=np.int32)
    idx_map = {j: i for i, j in enumerate(idx)}
    edges_unordered = np.genfromtxt("{}{}.cites".format(path, dataset),
                                    dtype=np.int32)
    edges = np.array(list(map(idx_map.get, edges_unordered.flatten())),
                     dtype=np.int32).reshape(edges_unordered.shape)
    adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                        shape=(labels.shape[0], labels.shape[0]),
                        dtype=np.float32)

    # build symmetric adjacency matrix
    adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)

    features = normalize(features)
    adj = normalize(adj + sp.eye(adj.shape[0]))

    idx_train = range(140)
    idx_val = range(200, 500)
    idx_test = range(500, 1500)
    
    labels = np.where(labels)[1]

    return adj, features, labels, idx_train, idx_val, idx_test

In [20]:
adj, features, labels, idx_train, idx_val, idx_test = load_data()

Loading cora dataset...


In [21]:
adj = adj.todense(); adj.shape

(2708, 2708)

In [22]:
features = features.todense(); features

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [23]:
labels

array([5, 0, 1, ..., 2, 6, 5])

In [24]:
idx_train

range(0, 140)

In [25]:
idx_val

range(200, 500)

In [26]:
idx_test

range(500, 1500)

In [27]:
known = np.array(list(idx_train)); known

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139])

In [28]:
gcn = GCN(adj, labels, method='cat', nl=3, init_feats=features, train_test=known)

In [29]:
init_preds = gcn.predict(); init_preds

tensor([2, 2, 2,  ..., 4, 4, 0])

In [30]:
(init_preds.numpy() == labels).mean()

0.16838995568685378

In [31]:
%time gcn.train(150)

CPU times: user 4min 5s, sys: 4.92 s, total: 4min 10s
Wall time: 23.9 s


In [32]:
preds = gcn.predict(); preds

tensor([5, 1, 1,  ..., 2, 6, 0])

In [33]:
(preds.numpy() == labels).mean()

0.4250369276218611

# Hand made dataset
This corresponds to the drawing in `proof_of_concept.jpg`

In [34]:
adj_m = [
        [1,1,1,0,0,0],
        [1,1,1,0,0,1],
        [1,1,1,1,1,0],
        [0,0,1,1,1,0],
        [0,0,1,1,1,1],
        [0,1,0,0,1,1]
                    ]

In [35]:
init_features = [
                [1, 5, 11, 0, 0, 0],
                [5, 1, 6, 0, 0, 13],
                [11, 6, 1, 7, 6, 0],
                [0, 0, 7, 1, 3, 0],
                [0, 0, 6, 3, 1, 6],
                [0, 13, 0, 0, 6, 1]
                                    ]

In [36]:
labels = [0, 0, 1, 1, 1, 1] # NOTE, only index 0, 2, and 3 are actually known
known = [0, 2, 3]

In [37]:
adj_m = np.array(adj_m)
init_feats = np.array(init_features, dtype=np.double)
labels = np.array(labels)
known = np.array(known)

In [38]:
np.place(init_feats, init_feats==0, 1000)

In [39]:
init_feats

array([[   1.,    5.,   11., 1000., 1000., 1000.],
       [   5.,    1.,    6., 1000., 1000.,   13.],
       [  11.,    6.,    1.,    7.,    6., 1000.],
       [1000., 1000.,    7.,    1.,    3., 1000.],
       [1000., 1000.,    6.,    3.,    1.,    6.],
       [1000.,   13., 1000., 1000.,    6.,    1.]])

In [40]:
# normalizes each row
norm_init = 1/init_feats.sum(axis=0); norm_init

array([0.00033146, 0.00049383, 0.00096993, 0.00033212, 0.00049603,
       0.00033113])

In [41]:
norm_init_feats = init_feats * norm_init.reshape(-1,1); norm_init_feats

array([[3.31455088e-04, 1.65727544e-03, 3.64600597e-03, 3.31455088e-01,
        3.31455088e-01, 3.31455088e-01],
       [2.46913580e-03, 4.93827160e-04, 2.96296296e-03, 4.93827160e-01,
        4.93827160e-01, 6.41975309e-03],
       [1.06692532e-02, 5.81959263e-03, 9.69932105e-04, 6.78952473e-03,
        5.81959263e-03, 9.69932105e-01],
       [3.32115576e-01, 3.32115576e-01, 2.32480903e-03, 3.32115576e-04,
        9.96346729e-04, 3.32115576e-01],
       [4.96031746e-01, 4.96031746e-01, 2.97619048e-03, 1.48809524e-03,
        4.96031746e-04, 2.97619048e-03],
       [3.31125828e-01, 4.30463576e-03, 3.31125828e-01, 3.31125828e-01,
        1.98675497e-03, 3.31125828e-04]])

In [42]:
gcn = GCN(adj_m, labels, method='cat', nl=3, init_feats=norm_init_feats, train_test=known)

In [43]:
preds_init = gcn.predict(); preds_init # randomly initial predictions

tensor([0, 1, 1, 0, 0, 0])

In [44]:
gcn.train(25)

In [45]:
preds = gcn.predict(); preds # pretty much what intuition suggests !!

tensor([0, 1, 1, 0, 0, 0])

In [46]:
gcn.epoch_stats # loss over epochs

[tensor(7.2859, dtype=torch.float64, grad_fn=<SumBackward0>),
 tensor(7.2443, dtype=torch.float64, grad_fn=<SumBackward0>),
 tensor(7.2034, dtype=torch.float64, grad_fn=<SumBackward0>),
 tensor(7.1629, dtype=torch.float64, grad_fn=<SumBackward0>),
 tensor(7.1226, dtype=torch.float64, grad_fn=<SumBackward0>),
 tensor(7.0826, dtype=torch.float64, grad_fn=<SumBackward0>),
 tensor(7.0429, dtype=torch.float64, grad_fn=<SumBackward0>),
 tensor(7.0034, dtype=torch.float64, grad_fn=<SumBackward0>),
 tensor(6.9643, dtype=torch.float64, grad_fn=<SumBackward0>),
 tensor(6.9247, dtype=torch.float64, grad_fn=<SumBackward0>),
 tensor(6.8830, dtype=torch.float64, grad_fn=<SumBackward0>),
 tensor(6.8415, dtype=torch.float64, grad_fn=<SumBackward0>),
 tensor(6.8005, dtype=torch.float64, grad_fn=<SumBackward0>),
 tensor(6.7597, dtype=torch.float64, grad_fn=<SumBackward0>),
 tensor(6.7193, dtype=torch.float64, grad_fn=<SumBackward0>),
 tensor(6.6793, dtype=torch.float64, grad_fn=<SumBackward0>),
 tensor(

# Continuous Dataset

In [47]:
adj_m = [
        [1,1,1,0,0,0],
        [1,1,1,0,0,1],
        [1,1,1,1,1,0],
        [0,0,1,1,1,0],
        [0,0,1,1,1,1],
        [0,1,0,0,1,1]
                    ]

In [48]:
init_features = [
                [1, 5, 11, 0, 0, 0],
                [5, 1, 6, 0, 0, 13],
                [11, 6, 1, 7, 6, 0],
                [0, 0, 7, 1, 3, 0],
                [0, 0, 6, 3, 1, 6],
                [0, 13, 0, 0, 6, 1]
                                    ]

In [49]:
labels = [-0.4, -0.2, 0, 0.1, 0.2, 0.3] # NOTE, only index 0, 2, and 3 are actually known
known = [0, 2, 3]

In [50]:
adj_m = np.array(adj_m)
init_feats = np.array(init_features, dtype=np.double)
labels = np.array(labels)
known = np.array(known)

In [51]:
np.place(init_feats, init_feats==0, 1000)

In [52]:
init_feats

array([[   1.,    5.,   11., 1000., 1000., 1000.],
       [   5.,    1.,    6., 1000., 1000.,   13.],
       [  11.,    6.,    1.,    7.,    6., 1000.],
       [1000., 1000.,    7.,    1.,    3., 1000.],
       [1000., 1000.,    6.,    3.,    1.,    6.],
       [1000.,   13., 1000., 1000.,    6.,    1.]])

In [53]:
# normalizes each row
norm_init = 1/init_feats.sum(axis=0); norm_init

array([0.00033146, 0.00049383, 0.00096993, 0.00033212, 0.00049603,
       0.00033113])

In [54]:
norm_init_feats = init_feats * norm_init.reshape(-1,1); norm_init_feats

array([[3.31455088e-04, 1.65727544e-03, 3.64600597e-03, 3.31455088e-01,
        3.31455088e-01, 3.31455088e-01],
       [2.46913580e-03, 4.93827160e-04, 2.96296296e-03, 4.93827160e-01,
        4.93827160e-01, 6.41975309e-03],
       [1.06692532e-02, 5.81959263e-03, 9.69932105e-04, 6.78952473e-03,
        5.81959263e-03, 9.69932105e-01],
       [3.32115576e-01, 3.32115576e-01, 2.32480903e-03, 3.32115576e-04,
        9.96346729e-04, 3.32115576e-01],
       [4.96031746e-01, 4.96031746e-01, 2.97619048e-03, 1.48809524e-03,
        4.96031746e-04, 2.97619048e-03],
       [3.31125828e-01, 4.30463576e-03, 3.31125828e-01, 3.31125828e-01,
        1.98675497e-03, 3.31125828e-04]])

In [55]:
gcn = GCN(adj_m, labels, method='cont', nl=3, init_feats=norm_init_feats, train_test=known)

In [56]:
preds_init = gcn.predict(); preds_init # randomly initial predictions

tensor([[3.9744],
        [4.3756],
        [5.7932],
        [5.0821],
        [5.3348],
        [3.5987]], dtype=torch.float64)

In [57]:
gcn.train(100)

In [58]:
preds_init = gcn.predict(); preds_init # not perfect but it captures the trend, only so much u can do w small dataset

tensor([[-0.3749],
        [-0.3240],
        [ 0.0775],
        [ 0.3513],
        [ 0.3049],
        [ 0.0840]], dtype=torch.float64)